In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

if False:
    # Turn off GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    gpu = False
else:
    gpu = True

from tensorflow import keras
from copy import deepcopy
import tensorflow as tf
from glob import glob
import numpy as np
import random

from utils.cityscapesSequence import CitySequence
from modelclass import Model
import models

if gpu:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 6))])

print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.7.0


In [21]:
GENERAL_PATH = os.path.join("model-saved", "cityscapes")

image_size = 256
batch_size = 2
lr = 1e-6
alpha = 1.


##### Dataset


In [22]:
def rain_file(DATA_DIR=None, TYPE=None):
    dirs = sorted(glob(os.path.join(DATA_DIR, "leftImg8bit_rain", TYPE, "*/*_leftImg8bit_rain_alpha_0.01_beta_0.005_dropsize_0.01_pattern_1.png")))

    list_dirs = []

    for dir in dirs:
        for i in range(len(dir)-1, -1, -1):
            if dir[i] == "/":
                i += 1
                break

        count = 0

        for j in range(len(dir[i:])):
            if dir[j] == "_":
                count += 1
            if count == 4:
                break
        list_dirs.append(glob(os.path.join(DATA_DIR, "/gtFine/gtFine", TYPE+"/*", dir[i:j] + "*_labelIds.png"))[0])

    dirs += sorted(glob(os.path.join(DATA_DIR, "leftImg8bit_rain", TYPE, "*/*_leftImg8bit_rain_alpha_0.02_beta_0.01_dropsize_0.005_pattern_5.png")))

    return [dirs, list_dirs*2]

In [23]:
def cityDataset(batch_size=1, do_remap=False, binary=False, gtFine=1):
    DATA_DIR = "../cityscape"

    if gtFine:
        DATA_TYPE = "gtFine"
        TYPE = "train"
    else:
        DATA_TYPE = "gtCoarse"
        TYPE = "train_extra"

    aux_list = rain_file(DATA_DIR, TYPE)

    x_train_dir = sorted(glob(os.path.join(DATA_DIR, DATA_TYPE, "leftImg8bit",
                                           TYPE, "*/*_leftImg8bit.png")))
    x_train_dir += sorted(glob(os.path.join(DATA_DIR, "leftImg8bit_foggyDBF",
                                           TYPE, "*/*_leftImg8bit_foggy_beta_0.02.png")))
    x_train_dir += sorted(glob(os.path.join(DATA_DIR, "leftImg8bit_foggyDBF",
                                           TYPE, "*/*_leftImg8bit_foggy_beta_0.005.png")))
    x_train_dir += aux_list[0]

    y_train_dir = sorted(glob(os.path.join(DATA_DIR, DATA_TYPE, DATA_TYPE,
                                           TYPE, "*/*_labelIds.png"))) * 3
    y_train_dir += aux_list[1]

    to_shuffle = list(zip(x_train_dir, y_train_dir))
    random.shuffle(to_shuffle)
    x_train_dir, y_train_dir = zip(*to_shuffle)

    TYPE = "val"

    aux_list = rain_file(DATA_DIR, TYPE)

    x_val_dir = sorted(glob(os.path.join(DATA_DIR, DATA_TYPE, "leftImg8bit",
                                         TYPE, "*/*_leftImg8bit.png")))
    x_val_dir += sorted(glob(os.path.join(DATA_DIR, "leftImg8bit_foggyDBF",
                                         TYPE, "*/*_leftImg8bit_foggy_beta_0.02.png")))
    x_val_dir += sorted(glob(os.path.join(DATA_DIR, "leftImg8bit_foggyDBF",
                                         TYPE, "*/*_leftImg8bit_foggy_beta_0.005.png"))) 
    x_val_dir += aux_list[0]

    y_val_dir = sorted(glob(os.path.join(DATA_DIR, DATA_TYPE, DATA_TYPE,
                                         TYPE, "*/*_labelIds.png"))) * 3 
    y_val_dir += aux_list[1]

    to_shuffle = list(zip(x_val_dir, y_val_dir))
    random.shuffle(to_shuffle)
    x_val_dir, y_val_dir = zip(*to_shuffle)

    x_train = CitySequence(x_dir=x_train_dir, y_dir=y_train_dir,
                           batch_size=batch_size, blur=0, crop=True,
                           image_size=image_size, horizontal_flip=True,
                           vertical_flip=True, brightness=0.1,
                           do_remap=do_remap, binary=binary)
    x_val = CitySequence(x_dir=x_val_dir, y_dir=y_val_dir,
                         batch_size=batch_size, blur=0, crop=True,
                         image_size=image_size, horizontal_flip=True,
                         vertical_flip=True, brightness=0,
                         do_remap=do_remap, binary=binary)

    print(len(x_train_dir), len(y_train_dir), x_train)
    print(len(x_val_dir), len(y_val_dir), x_val)
    print(x_train.num_classes, x_val.num_classes)

    return x_train, x_val


x_train, x_val = cityDataset(
    batch_size=batch_size, do_remap=False, binary=True, gtFine=1)
num_classes = x_train.num_classes


9449 9449 <utils.cityscapesSequence.CitySequence object at 0x7efe8b331c10>
1566 1566 <utils.cityscapesSequence.CitySequence object at 0x7efe88f66290>
2 2


##### Train

In [24]:
MODEL_NAME = "binary_fog_rain.hdf5"
MODEL_PATH = os.path.join(GENERAL_PATH, MODEL_NAME)

model = Model(image_size=image_size, num_classes=num_classes, alpha=alpha,
              generate_model=models.generate_model)

model.summary()
model.compile()

Model: "DeepLabV3Plus.py"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                                  

In [25]:
model.train(x_train=x_train, x_val=x_val, batch_size=batch_size,
            epochs=200, SAVE_PATH=MODEL_PATH, monitor="val_loss")

Epoch 1/200
4725/4725 [==============================] - ETA: 0s - loss: 3.7557 - accuracy: 0.9587 - precision: 0.9516 - recall: 0.9486 - f1Score: 0.9486 - mIOU: 0.9195 - jaccardDistance: 0.8046 - dice_accuracy: 0.9569
Epoch 00001: val_loss improved from inf to 0.71706, saving model to model-saved/cityscapes/binary_fog_rain.hdf5


/home/mrbatman/.local/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


4725/4725 [==============================] - 927s 196ms/step - loss: 3.7557 - accuracy: 0.9587 - precision: 0.9516 - recall: 0.9486 - f1Score: 0.9486 - mIOU: 0.9195 - jaccardDistance: 0.8046 - dice_accuracy: 0.9569 - val_loss: 0.7171 - val_accuracy: 0.5728 - val_precision: 0.7859 - val_recall: 0.0067 - val_f1Score: 0.0132 - val_mIOU: 0.2891 - val_jaccardDistance: 5.9033 - val_dice_accuracy: 0.5783 - lr: 7.0000e-04
Epoch 2/200
4725/4725 [==============================] - ETA: 0s - loss: 0.3527 - accuracy: 0.9710 - precision: 0.9650 - recall: 0.9638 - f1Score: 0.9638 - mIOU: 0.9421 - jaccardDistance: 0.5628 - dice_accuracy: 0.9707
Epoch 00002: val_loss improved from 0.71706 to 0.38522, saving model to model-saved/cityscapes/binary_fog_rain.hdf5
4725/4725 [==============================] - 926s 196ms/step - loss: 0.3527 - accuracy: 0.9710 - precision: 0.9650 - recall: 0.9638 - f1Score: 0.9638 - mIOU: 0.9421 - jaccardDistance: 0.5628 - dice_accuracy: 0.9707 - val_loss: 0.3852 - val_accurac

In [ ]:
MODEL_NAME = "binary_noSkip_fog_rain.hdf5"
MODEL_PATH = os.path.join(GENERAL_PATH, MODEL_NAME)

model = Model(image_size=image_size, num_classes=num_classes, alpha=alpha,
              generate_model=models.generate_model_no_skip)

model.summary()
model.compile()

Model: "DeepLabV3Plus.py"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                                  

In [ ]:
model.train(x_train=x_train, x_val=x_val, batch_size=batch_size,
            epochs=200, SAVE_PATH=MODEL_PATH, monitor="val_loss")

#### **Quantization**


##### TFLITE Converter


In [5]:
from random import shuffle
from glob import glob
import os
import sys
import importlib


def representative_dataset_gen():
    DATA_DIR = "../cityscape/gtFine"
    IMAGE_PATH = os.path.join(
        DATA_DIR, "leftImg8bit/train/*/*_leftImg8bit.png")
    images = glob(IMAGE_PATH)
    shuffle(images)
    images = images[0:75]

    for img in images:
        aux = tf.io.read_file(img)
        aux = tf.image.decode_image(aux, channels=3)
        aux.set_shape([None, None, 3])
        aux = tf.image.resize(images=aux,
                              size=[256, 256]) / 127.5 - 1
        yield [tf.reshape(aux, (1, aux.shape[0], aux.shape[1], aux.shape[2]))]

In [8]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join(GENERAL_PATH, MODEL_NAME)

model = Model(image_size=image_size, num_classes=num_classes, alpha=alpha,
              generate_model=models.generate_model, with_argmax=True)

model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representative_dataset_gen, supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS])

1 of 1 : TFLITE_BUILTINS
INFO:tensorflow:Assets written to: /tmp/tmp_kv44pdq/assets


INFO:tensorflow:Assets written to: /tmp/tmp_kv44pdq/assets
2022-05-29 16:49:52.198315: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-05-29 16:49:52.198342: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.


Modelo Salvo em model-saved/cityscapes/TFLITE_BUILTINS_binary.tflite


fully_quantize: 0, inference_type: 6, input_inference_type: 0, output_inference_type: 0


In [ ]:
MODEL_NAME = "binary_noSkip.hdf5"
MODEL_PATH = os.path.join(GENERAL_PATH, MODEL_NAME)

model = Model(image_size=image_size, num_classes=num_classes, alpha=alpha,
              generate_model=models.generate_model_no_skip_argmax)

model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representative_dataset_gen)